In [3]:
import cv2  # Importing the OpenCV library for computer vision
import mediapipe as mp  # Importing the Mediapipe library for hand tracking
import time  # Importing the time library for delays
import serial  # Importing the serial library for communication with Arduino

comport = 'COM3'  # Specify the COM port for Arduino
baudrate = 9600  # Specify the baud rate for serial communication

# Establish serial communication with Arduino
arduino = serial.Serial(comport, baudrate)
time.sleep(2)  # Allow time for Arduino to reset

mp_draw = mp.solutions.drawing_utils  # Utility functions for drawing landmarks on the image
mp_hand = mp.solutions.hands  # Hand tracking solution from Mediapipe
tipIds = [4, 8, 12, 16, 20]  # List of IDs for the fingertips

video = cv2.VideoCapture(0)  # Create a video capture object to read frames from the default camera

with mp_hand.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    # Start a Mediapipe Hands object with minimum confidence thresholds for detection and tracking
    while True:
        ret, image = video.read()  # Read a frame from the video capture
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert the color space of the image to RGB
        image.flags.writeable = False  # Make the image read-only to improve performance
        results = hands.process(image)  # Process the hand tracking on the image
        image.flags.writeable = True  # Make the image writable again
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert the color space back to BGR

        lmList = []  # List to store landmark coordinates

        if results.multi_hand_landmarks:
            # If hand landmarks are detected in the frame
            for hand_landmark in results.multi_hand_landmarks:
                myHands = results.multi_hand_landmarks[0]
                for id, lm in enumerate(myHands.landmark):
                    h, w, c = image.shape  # Get the height, width, and number of channels of the image
                    cx, cy = int(lm.x * w), int(lm.y * h)  # Calculate the pixel coordinates of the landmark
                    lmList.append([id, cx, cy])  # Append the landmark ID and coordinates to the list
                mp_draw.draw_landmarks(image, hand_landmark, mp_hand.HAND_CONNECTIONS)
                # Draw landmarks and connections on the image

        fingers = []  # List to store finger states

        if len(lmList) != 0:
            # If landmarks are detected
            if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
                fingers.append(1)  # Append 1 if the thumb is open
            else:
                fingers.append(0)  # Append 0 if the thumb is closed

            for id in range(1, 5):
                # Loop through the other fingers
                if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                    fingers.append(1)  # Append 1 if the finger is open
                else:
                    fingers.append(0)  # Append 0 if the finger is closed

            total = fingers.count(1)  # Count the number of open fingers

            # Calculate finger pinch distance
            pinch_distance = abs(lmList[tipIds[0]][1] - lmList[tipIds[2]][1])

            # Map pinch distance to intensity range (0-255)
            intensity = int((pinch_distance / 200) * 255)
            intensity = min(max(intensity, 0), 255)  # Ensure intensity is within the valid range

            # Draw dots on the screen
            cv2.circle(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), 5, (0, 0, 255), cv2.FILLED)
            cv2.circle(image, (lmList[tipIds[2]][1], lmList[tipIds[2]][2]), 5, (0, 0, 255), cv2.FILLED)

            # Draw a line between the dots
            cv2.line(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), (lmList[tipIds[2]][1], lmList[tipIds[2]][2]), (0, 0, 255), 2)

            arduino.write(str(intensity).encode() + b'\n')  # Send intensity to Arduino

        cv2.imshow("Frame", image)  # Display the image with drawn landmarks

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Close on pressing 'q' key
            break

# Close the serial connection
arduino.close()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
pip install pyserial

In [ ]:
import cv2
import mediapipe as mp
import time
import serial

comport = 'COM3'
baudrate = 9600

# Establish serial communication with Arduino
arduino = serial.Serial(comport, baudrate)
time.sleep(2)  # Allow time for Arduino to reset

mp_draw = mp.solutions.drawing_utils
mp_hand = mp.solutions.hands
tipIds = [4, 8, 12, 16, 20]

video = cv2.VideoCapture(0)

with mp_hand.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while True:
        ret, image = video.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lmList = []
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                myHands = results.multi_hand_landmarks[0]
                for id, lm in enumerate(myHands.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmList.append([id, cx, cy])
                mp_draw.draw_landmarks(image, hand_landmark, mp_hand.HAND_CONNECTIONS)
        fingers = []
        if len(lmList) != 0:
            if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
            for id in range(1, 5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            total = fingers.count(1)

            # Calculate finger pinch distance
            pinch_distance = abs(lmList[tipIds[0]][1] - lmList[tipIds[1]][1])

            # Map pinch distance to intensity range (0-255)
            intensity = int((pinch_distance / 200) * 255)
            intensity = min(max(intensity, 0), 255)  # Ensure intensity is within the valid range

            # Draw dots on the screen
            cv2.circle(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), 5, (0, 0, 255), cv2.FILLED)
            cv2.circle(image, (lmList[tipIds[1]][1], lmList[tipIds[1]][2]), 5, (255, 0, 0), cv2.FILLED)

            # Draw a line between the dots
            cv2.line(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), (lmList[tipIds[1]][1], lmList[tipIds[1]][2]), (255, 0, 0), 2)
            cv2.line(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), (lmList[tipIds[1]][1], lmList[tipIds[1]][2]), (255, 0, 0), 2)

            arduino.write(str(intensity).encode() + b'\n')  # Send intensity to Arduino

        cv2.imshow("Frame", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Close on pressing 'q' key
            break

# Close the serial connection
arduino.close()
cv2.destroyAllWindows()


In [3]:
import cv2
import mediapipe as mp
import time
import serial

comport = 'COM3'
baudrate = 9600

# Establish serial communication with Arduino
arduino = serial.Serial(comport, baudrate)
time.sleep(1)  # Allow time for Arduino to reset

mp_draw = mp.solutions.drawing_utils
mp_hand = mp.solutions.hands
tipIds = [4, 8, 12, 16, 20]

video = cv2.VideoCapture(0)

with mp_hand.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while True:
        ret, image = video.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lmList = []
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                myHands = results.multi_hand_landmarks[0]
                for id, lm in enumerate(myHands.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmList.append([id, cx, cy])
                mp_draw.draw_landmarks(image, hand_landmark, mp_hand.HAND_CONNECTIONS)
        fingers = []
        if len(lmList) != 0:
            if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
            for id in range(1, 5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            total = fingers.count(1)

            # Calculate finger pinch distance
            pinch_distance = abs(lmList[tipIds[0]][1] - lmList[tipIds[2]][1])

            # Map pinch distance to intensity range (0-255)
            intensity = int((pinch_distance / 200) * 255)
            intensity = min(max(intensity, 0), 255)  # Ensure intensity is within the valid range

            # Draw dots on the screen
            cv2.circle(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), 5, (0, 0, 255), cv2.FILLED)
            cv2.circle(image, (lmList[tipIds[2]][1], lmList[tipIds[2]][2]), 5, (255, 0, 0), cv2.FILLED)

            # Draw a line between the dots
            cv2.line(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), (lmList[tipIds[2]][1], lmList[tipIds[2]][2]), (255, 0, 0), 2)

            arduino.write(str(intensity).encode() + b'\n')  # Send intensity to Arduino

        cv2.imshow("Frame", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Close on pressing 'q' key
            break

# Close the serial connection
arduino.close()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import time
import serial

comport = 'COM3'
baudrate = 9600

# Establish serial communication with Arduino
arduino = serial.Serial(comport, baudrate)
time.sleep(1)  # Allow time for Arduino to reset

mp_draw = mp.solutions.drawing_utils
mp_hand = mp.solutions.hands
tipIds = [4, 8, 12, 16, 20]

video = cv2.VideoCapture(0)
video.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Set width to 640 pixels
video.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Set height to 480 pixels

with mp_hand.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while True:
        ret, image = video.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lmList = []
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                myHands = results.multi_hand_landmarks[0]
                for id, lm in enumerate(myHands.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmList.append([id, cx, cy])
                mp_draw.draw_landmarks(image, hand_landmark, mp_hand.HAND_CONNECTIONS)
        fingers = []
        if len(lmList) != 0:
            if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
            for id in range(1, 5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            total = fingers.count(1)

            # Calculate finger pinch distance
            pinch_distance = abs(lmList[tipIds[0]][1] - lmList[tipIds[2]][1])

            # Map pinch distance to intensity range (0-255)
            intensity = int((pinch_distance / 200) * 255)
            intensity = min(max(intensity, 0), 255)  # Ensure intensity is within the valid range

            # Draw dots on the screen
            for i in tipIds:
                cv2.circle(image, (lmList[i][1], lmList[i][2]), 5, (0, 0, 255), cv2.FILLED)

            # Draw a line between the dots
            cv2.line(image, (lmList[tipIds[0]][1], lmList[tipIds[0]][2]), (lmList[tipIds[1]][1], lmList[tipIds[1]][2]), (255, 0, 0), 2)

            arduino.write(str(intensity).encode() + b'\n')  # Send intensity to Arduino

        cv2.imshow("Frame", image)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Close on pressing 'q' key
            break

# Close the serial connection
arduino.close()
cv2.destroyAllWindows()
